In [2]:

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

from sympy.solvers import solve
from sympy import Symbol
import math
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from numpy import linalg as LA

import time

In [3]:
def function(y,E,theta,eta,w,delta,alpha,s_0,b,rho,tv):
    
    mu = 25
    W,L,p,n,lambda_ = y 
    
    A = p*n/w
    
    #dot_W = x[0]*rho*((mu/x[2]-1)+float(np.random.normal(0,0.22,1)))
    dot_W = W*rho*((mu/p-1))
    #dot_W = 0
    dot_L = (theta*(lambda_*(A-L)-A))
    #dot_L = theta*(lambda_*(E*lambda_-x[1])-E*lambda_ )
    
    dot_p = (p* (1-n) * dot_W / W + eta*(w-W)*(E-A+L)+ w*dot_L ) / (1-n*w-(1-n)*W)
    #dot_p = ( eta*(w-x[0])*(E-A+x[1])+ w*dot_L ) 
    # / (1-x[3]*w-(1-x[3])*x[0])
    dot_n = n* (-(1-w)*dot_p/p + 1/A*(eta*(E-A+L)+dot_L))

    #dot_s = -delta*(x[4]) + delta * (np.log(1-dot_p/x[2]))**2
    #dot_s = -delta*(x[4]) + delta * (np.log(1-dot_p/x[2]))**2
    #dot_s = -delta*(s) + delta * tv**2*(-dot_p/p)**2
    a1 = w/p * theta * (A - L) / (1-n*w-(1-n)*W)
    a0 = (theta*n-(1-n)*rho*(mu/p-1)-eta/p*(w-W)*(E-A+L)) / (1-n*w-(1-n)*W)

    dot_lambda = lambda_*delta/2 * (1- (lambda_/alpha)**2 * (s_0 + tv**2*(a1*lambda_-a0)**2))

    return([dot_W, dot_L, dot_p, dot_n, dot_lambda])
   

In [4]:
def sys(iv, dt, time,function,E,theta,eta,w,delta,alpha,s_0,b,rho,tv, scale_v):
    n = int(time/dt)
    t = np.linspace(0,time,n)
    x = np.array([[ i for i in iv]]*n)
    n_ord = len(iv)
    
    #scale_v = np.array([w,2*theta,mu,w,theta])
    
    #scale_v = np.array([1,1,1,1,1])
    a = iv/scale_v

    
    
    for i in range(1,n):
        X = function(a*scale_v,E,theta,eta,w,delta,alpha,s_0,b,rho,tv)
        X[0] = X[0] 
        a = a + (np.array(X)*dt)/scale_v 
        #print('a = ',a)
        x[i] = a
    return np.array(x)


In [26]:
rho= 0.1
#rho = 100

E = 2
mu = 25
theta = 10
eta = 10
w = 0.5

#tv = 10**-1
#alpha = 10**-2
#s_0 = 10**-6


tv = 10**-1
alpha = 0.01
s_0 = 10**-6
#s_0=0

b = -0.5

In [29]:
T = 100
dt = 10**-4
delta = 0.5

start_time = time.time()

p0 = 25
L0 = 20
#x= sys([0.2,L0,p0,0.5,1], dt,T,function,E,theta,eta,w,delta,alpha,s_0,b,rho,tv)
#ss = (alpha/0.1)**2-10**-6

scale_v = np.array([0.449,16.30,30.39,0.30,9.178])
scale_v = np.array([0.08367570934367924,18.025539887517258,13.782762702144932,0.29302765940913145,9.057653504809467])

x= sys([0.3,-4,30,1,0.1], dt,T,function,E,theta,eta,w,delta,alpha,s_0,b,rho,tv, scale_v)

print("--- %s seconds ---" % (time.time() - start_time))


n = int(T/dt)
t = np.linspace(0,T,n)

--- 15.07826542854309 seconds ---


In [30]:
fig = plt.figure(figsize=(5,5))
fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(5,1,1)
ax2 = fig.add_subplot(5,1,2)
ax3 = fig.add_subplot(5,1,3)
ax4 = fig.add_subplot(5,1,4)
ax5 = fig.add_subplot(5,1,5)

n = int(T/dt)
t = np.linspace(0,T,n)

#lambda_ = alpha * (x[:,4] + s_0)**b
w = 0.5
#A = x[:,2]*x[:,3]/w

T1 = 100
T2 = 300
#xx= np.array([x[500*i] for i in range(int(len(x)//1000))])
#t = np.array([t[500*i] for i in range(int(len(t)//1000))])
lambda_ = x[:,4]

tau = 1
#ax1.plot(t,10*(A-x[:,1])/x[:,2], 'r-',label ='W')
ax1.plot(tau*t,x[:,0], 'k-',label ='W')
ax1.set_ylabel(r'$W$')


#ax2.plot(t,(1-1/lambda_)*x[:,2]*x[:,3]/w, 'k',label ='L',alpha = 0.2)
ax2.plot(tau*t,x[:,1], 'k-',label ='L')
#ax2.plot(t,x[:,1]-(lambda_-1)*E, 'b-',label ='L')
ax2.set_ylabel(r'$L$')



###### AX3 P ######

ax3.plot(tau*t,x[:,2], 'k-',label ='p')



###### AX4 ######

ax4.plot(tau*t,x[:,3], 'k-',label ='n')
#ax4.plot(t,w/x[:,2] * (E+x[:,1]), 'r--',alpha=0.4)
ax4.set_ylabel(r'$n$')


#ax5.plot(t,A/E, 'k--',label ='s',alpha=0.5)
ax5.plot(tau*t,lambda_, 'k-',label ='s')

#ax5.plot(t,(np.log(1-theta*lambda_*(E*(lambda_-1)-x[:,1]) / (x[:,1]+10)))**2, 'g--',label ='s')
ax5.set_ylabel(r'$ \lambda $')
ax5.set_xlabel(r'$ t $')

#ax1.set_ylim([0,1])
#ax2.set_ylim([0,1])
#ax3.set_ylim([0,1])
#ax4.set_ylim([0,1])
#ax5.set_ylim([0,1])

plt.grid()
plt.show()


In [411]:
### PLOT AGAINST SLOW MANI CANDIDATES

In [39]:
W,L,p,n,lambda_ = np.transpose(x * scale_v)


In [41]:
plt.figure

A = n*p/w

plt.plot(t,L/scale_v[1])
plt.plot(t,A*(1-1/lambda_)/scale_v[1])

In [32]:
fig = plt.figure(figsize=(5,5))
fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(5,1,1)
ax2 = fig.add_subplot(5,1,2)
ax3 = fig.add_subplot(5,1,3)
ax4 = fig.add_subplot(5,1,4)
ax5 = fig.add_subplot(5,1,5)

n = int(T/dt)
t = np.linspace(0,T,n)

# variable letter
w = 0.5
W,L,p,n,lambda_ = np.transpose(x * scale_v)

A = n*p/w
#a1 = w/p * theta * (A - L) / (1-n*w-(1-n)*W)
#a0 = (theta*n-(1-n)*rho*(mu/p-1)-eta/p*(w-W)*(E-A+L)) / (1-n*w-(1-n)*W)
tv,s_0,alpha =  0.1, 10**-6, 0.01

T1,T2 = 3500,4000
T1,T2 = 3585,7360
#T1,T2 = 3585,3620
T1,T2 = 5000,18000
T1,T2 = 300000,1000000
#T1,T2 = 17000,17200

size = 1.1

#ax1.plot(t,10*(A-x[:,1])/x[:,2], 'r-',label ='W')
#ax1.plot(t[T1:T2],W[T1:T2], 'k-',label ='W')
ax1.plot(t[T1:T2],A[T1:T2], 'k-',label ='W')


num = (1-n)*rho*mu + eta*(w-W)*(E+L) - w * theta*lambda_*L
dem =  (1-n)*rho + eta*n*(w-W)/w + theta*n*(1-lambda_)

a_ = eta*(w-W) - theta * (lambda_-1)*(w-W)
b_ = - eta*((E+L)/p *w*(w-W) + (w**2-2*w*W+W)/w) + theta * ((lambda_-1)*(w-W) + w*L*lambda_ / p *(w-W)) + (1-w)*rho*(mu/p-1)
c_ = eta*(E+L)/p *(w**2-2*w*W+W) - theta * w * L *lambda_ /p * (w-W) - (1-w)*rho*(mu/p-1)

DELTA = b_**2 - 4*a_*c_
D_ = [i*(i>0) for i in DELTA]
d_ = np.sqrt(D_)

#ax1.plot(t[T1:T2],(1/(w*lambda_[T1:T2]))*num[T1:T2]/dem[T1:T2] * (-b_[T1:T2] + d_[T1:T2] )/ (2*a_[T1:T2]),'b--',label ='n',alpha=0.5)
#ax1.plot(t[T1:T2],(1/(w*lambda_[T1:T2]))*num[T1:T2]/dem[T1:T2] * (-b_[T1:T2] - d_[T1:T2] )/ (2*a_[T1:T2]),'g--',label ='n',alpha=0.5)

ax1.plot(t[T1:T2],(1/(w))*num[T1:T2]/dem[T1:T2] * (-b_[T1:T2] + d_[T1:T2] )/ (2*a_[T1:T2]),'b--',label ='n',alpha=0.5)
ax1.plot(t[T1:T2],(1/(w))*num[T1:T2]/dem[T1:T2] * (-b_[T1:T2] - d_[T1:T2] )/ (2*a_[T1:T2]),'g--',label ='n',alpha=0.5)

#ax1.plot(t[T1:T2],0*A[T1:T2]-E, 'b--',label ='W')
ax1.set_ylabel(r'$W$')
#ax1.set_ylim([-5,50])

#### AX2 #####

ax2.plot(t[T1:T2],L[T1:T2], 'k-',label ='L')
L_sm = A*(1-1/lambda_)
ax2.plot(t[T1:T2], L_sm[T1:T2]//scale_v[1], 'b--',label ='L',alpha=0.5)
ax2.set_ylabel(r'$L$')

#ax2.set_ylim([np.min(L[T1:T2])*size,np.max(L[T1:T2])*size])


###### AX3 P ######

ax3.plot(t[T1:T2],p[T1:T2], 'k-',label ='p')

num = (1-n)*rho*mu + eta*(w-W)*(E+L) - w * theta*lambda_*L
dem =  (1-n)*rho + eta*n*(w-W)/w + theta*n*(1-lambda_)

ax3.plot(t[T1:T2],num[T1:T2]/dem[T1:T2] /scale_v[2] , 'b--',label ='p',alpha=0.5)
#ax3.set_ylim([np.min(p[T1:T2])/size,np.max(p[T1:T2])*size])

###### AX4 ######

ax4.plot(t[T1:T2],n[T1:T2], 'k-',label ='n')
ax4.set_ylabel(r'$n$')
a_ = eta*(w-W) - theta * (lambda_-1)*(w-W)
b_ = - eta*((E+L)/p *w*(w-W) + (w**2-2*w*W+W)/w) + theta * ((lambda_-1)*(w-W) + w*L*lambda_ / p *(w-W)) + (1-w)*rho*(mu/p-1)
c_ = eta*(E+L)/p *(w**2-2*w*W+W) - theta * w * L *lambda_ /p * (w-W) - (1-w)*rho*(mu/p-1)

DELTA = b_**2 - 4*a_*c_
D_ = [i*(i>0) for i in DELTA]
d_ = np.sqrt(D_)

ax4.plot(t[T1:T2],(-b_[T1:T2] + d_[T1:T2] )/ (2*a_[T1:T2]) /scale_v[3],'b--',label ='n',alpha=0.5)
ax4.plot(t[T1:T2],(-b_[T1:T2] - d_[T1:T2] )/ (2*a_[T1:T2])/scale_v[3],'g--',label ='n',alpha=0.5)
#ax4.set_ylim([-0.1,np.max(n[T1:T2])*size])

#ax5.plot(t,A/E, 'k--',label ='s',alpha=0.5)

#ax5.plot(t,(np.log(1-theta*lambda_*(E*(lambda_-1)-x[:,1]) / (x[:,1]+10)))**2, 'g--',label ='s')
ax5.set_ylabel(r'$ \lambda $')
ax5.set_xlabel(r'$ t $')

from numpy import arange
from numpy import meshgrid
d = 0.01
xrange = arange(t[T1], t[T2], dt)
yrange = arange(-1, 11, d)
X, Y = meshgrid(xrange,yrange)
# F is one side of the equation, G is the other
#F = delta*X**3 + X**2 * Y - delta/4 * Y**2  # Cusp with approximation

#F = -b*delta*Y*(1-Y**2/alpha**2 * (s_0 + tv**2*(a1[(X/dt).astype(int)]*Y-a0[(X/dt).astype(int)])**2))
#F = -b*delta*Y*(1-Y**2/alpha**2 * (s_0 + tv**2*(a1[(X/dt).astype(int)]*Y-a0[(X/dt).astype(int)])**2))



#### #ALTER ####
a_ = (1-w)*rho*(mu/p-1) + eta * (w-W)*(1-w)/ (lambda_*w)
b_ = -(1-w)*(rho*(mu/p-1)  + eta*(w-W)*E/p) - eta/lambda_
c_ = eta*E*w/p
DELTA = b_**2 - 4*a_*c_
D_ = [i*(i>0) for i in DELTA]
d_ = np.sqrt(D_)

nmoins = (-b_- d_ )/ (2*a_)
#n = nmoins
A = p*n/w
#L = A*(1-1/lambda_)


#a1 = w/p[(X/dt).astype(int)] * theta * (A[(X/dt).astype(int)] - L[(X/dt).astype(int)]) / (1-n[(X/dt).astype(int)]*w-(1-n[(X/dt).astype(int)])*W[(X/dt).astype(int)])
#a0 = theta*n[(X/dt).astype(int)]-(1-n[(X/dt).astype(int)])*rho*(mu/p[(X/dt).astype(int)]-1)-eta/p[(X/dt).astype(int)]*(w-W[(X/dt).astype(int)])*(E-A[(X/dt).astype(int)]+L[(X/dt).astype(int)]) / (1-n[(X/dt).astype(int)]*w-(1-n[(X/dt).astype(int)])*W[(X/dt).astype(int)])

a1 = w/p * theta * (A - L) / (1-n*w-(1-n)*W)
a0 = (theta*n-(1-n)*rho*(mu/p-1)-eta/p*(w-W)*(E-A+L)) / (1-n*w-(1-n)*W)

G = -b*delta*Y*(1-Y**2/alpha**2 * (s_0 + tv**2*(a1[(X/dt).astype(int)]*Y-a0[(X/dt).astype(int)])**2))

#plt.grid()
#ax5.contour(X, Y, F , [0])
ax5.contour(X, Y, G , [0])
ax5.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',alpha=0.5)
#ax5.set_xlim([34,75.5])
#ax5.set_xlim([35.834,36.208])

plt.show()


IndexError: index 1000000 is out of bounds for axis 0 with size 1000000

In [388]:
#### PLOT THE DOT ################


fig = plt.figure(figsize=(5,5))

T1,T2 = 5585,20360


dot_W = x[:,0]*rho*((mu/x[:,2]-1))
dot_L = theta*(lambda_*(A-x[:,1])-A)
dot_p = (x[:,2]* (1-x[:,3]) * dot_W / x[:,0] + eta*(w-x[:,0])*(E-A+x[:,1])+ w*dot_L ) / (1-x[:,3]*w-(1-x[:,3])*x[:,0])
dot_n = x[:,3]* (-(1-w)*dot_p/x[:,2] + 1/A*(eta*(E-A+x[:,1])+dot_L))

dot_lambda_ = lambda_*delta/2 * (1- (lambda_/alpha)**2 * (s_0 + tv**2*(a1*lambda_-a0)**2))

plt.plot(t[T1:T2],dot_W[T1:T2]/(np.max(W[T1:T2])-np.min(W[T1:T2])),label='W',alpha=0.5)
,
#ax1.plot(t[T1:T2],0*A[T1:T2]-E, 'b--',label ='W')

#### AX2 #####

plt.plot(t[T1:T2],dot_L[T1:T2]/ Lampl, '-',label ='L',alpha=0.5)

###### AX3 P ######

plt.plot(t[T1:T2],dot_p[T1:T2]/ pampl, '-',label ='p',alpha=0.5)

#ax3.set_ylim([0,40])

###### AX4 ######

plt.plot(t[T1:T2],dot_n[T1:T2]/nampl, '-',label ='n',alpha=0.5)



plt.plot(t[T1:T2],dot_lambda_[T1:T2]/lambdaampl, '-',label ='s',alpha=0.5)

plt.ylim([-1,1])
plt.legend()

In [ ]:
plt.figure()
T1,T2 = 3570,4860

plt.plot(t[T1:T2],a0[T1:T2], '-',label ='s')
plt.plot(t[T1:T2],a1[T1:T2], '-',label ='s')
plt.plot(t[T1:T2],lambda_[T1:T2], '-',label ='s',alpha=0.1)

In [148]:
print(np.argmin(a0[T1:T2]),np.argmax(a1[T1:T2]))

2820 2804


In [149]:
3570 +86

3656

In [150]:
plt.figure()
T1,T2 = 3585,7360
#T1,T2 = 3585,3620

#ax1.plot(t,10*(A-x[:,1])/x[:,2], 'r-',label ='W')
#ax1.plot(t[T1:T2],W[T1:T2], 'k-',label ='W')
plt.plot(t[T1:T2],A[T1:T2]-L[T1:T2], 'k-',label ='W')


num = (1-n)*rho*mu + eta*(w-W)*(E+L) - w * theta*lambda_*L
dem =  (1-n)*rho + eta*n*(w-W)/w + theta*n*(1-lambda_)

a_ = eta*(w-W) - theta * (lambda_-1)*(w-W)
b_ = - eta*((E+L)/p *w*(w-W) + (w**2-2*w*W+W)/w) + theta * ((lambda_-1)*(w-W) + w*L*lambda_ / p *(w-W)) + (1-w)*rho*(mu/p-1)
c_ = eta*(E+L)/p *(w**2-2*w*W+W) - theta * w * L *lambda_ /p * (w-W) - (1-w)*rho*(mu/p-1)

DELTA = b_**2 - 4*a_*c_
D_ = [i*(i>0) for i in DELTA]
d_ = np.sqrt(D_)

plt.plot(t[T1:T2],(1/(w*lambda_[T1:T2]))*num[T1:T2]/dem[T1:T2] * (-b_[T1:T2] + d_[T1:T2] )/ (2*a_[T1:T2]),'b--',label ='n',alpha=0.5)
plt.plot(t[T1:T2],(1/(w*lambda_[T1:T2]))*num[T1:T2]/dem[T1:T2] * (-b_[T1:T2] - d_[T1:T2] )/ (2*a_[T1:T2]),'g--',label ='n',alpha=0.5)

#plt.ylim([-5,10])

In [151]:
plt.figure()
T1,T2 = 3500,4000

plt.plot(t[T1:T2],A[T1:T2]- L[T1:T2], 'k-',label ='L')
L_sm = A*(1-1/lambda_)
#plt.plot(t[T1:T2], L_sm[T1:T2-], 'b--',label ='L',alpha=0.5)

In [152]:
# amplitude

In [32]:
def min_max(l):
    return(np.min(l),np.max(l))

In [35]:


Lm,LM = min_max(L[T1:T2])
pm,pM = min_max(p[T1:T2])
nm,nM = min_max(n[T1:T2])
lambdam,lambdaM = min_max(lambda_[T1:T2])

Lampl = -Lm + LM
pampl = -pm + pM
nampl = -nm + nM
lambdaampl  = -lambdam + lambdaM


TypeError: 'int' object is not subscriptable

In [442]:
from matplotlib import ticker, cm, colors

In [156]:
dt

0.01

In [ ]:
plt.figure()
plt.plot(t[T1:T2],a0[T1:T2])

In [21]:
# plot lambda slow manifold

from numpy import arange
from numpy import meshgrid

fig, ax = plt.subplots()




d = 0.01
T1,T2 = 3500,6532

xrange = arange(t[T1], t[T2], dt)
yrange = arange(0, 11, d)
X, Y = meshgrid(xrange,yrange)

# F is one side of the equation, G is the other

G = alpha**2 - Y**2*(3*s_0+3*tv**2*a0[(X/dt).astype(int)]**2) + Y**3 * 8*a0[(X/dt).astype(int)]*a1[(X/dt).astype(int)]*tv**2 - Y**4 *5*tv**2*a1[(X/dt).astype(int)]**2

#G =  -b*delta*Y*(1-Y**2/alpha**2 * (s_0 + tv**2*(a1[(X/dt).astype(int)]*Y-a0[(X/dt).astype(int)])**2))

#cs = ax.contourf(X, Y, G, locator=ticker.LogLocator(), cmap=cm.PuBu_r)

#cs = ax.contourf(X, Y, G)

cs = ax.contourf(X, Y, G, [-1,-0.1,0,0.1,1])

cbar = fig.colorbar(cs)

#F = delta*X**3 + X**2 * Y - delta/4 * Y**2  # Cusp with approximation
F = -b*delta*Y*(1-Y**2/alpha**2 * (s_0 + tv**2*(a1[(X/dt).astype(int)]*Y-a0[(X/dt).astype(int)])**2))

plt.grid()
plt.contour(X, Y, F , [0])

plt.plot(t[T1:T2],lambda_[T1:T2], 'r+--',label ='s')


In [14]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.contour(X,Y,G)

In [18]:
np.max(G)

49.73998242714025

In [9]:
#num = (1-n)*rho*mu+eta*(w-W)*(E+L)-w*theta*lambda_*L
#denom = (1-n)*rho + eta*(w-W)*n/w -theta*n*(lambda_-1)
num = (1-n)*rho*mu+eta*(w-W)*(E)
denom = (1-n)*rho - eta*(w-W)*n/ (w* lambda_)

A = x[:,2]*x[:,3]/w
mu = 25
#dot_W = x[0]*rho*((mu/x[2]-1)+float(np.random.normal(0,0.22,1)))
dot_W = x[:,0]*rho*((mu/x[:,2]-1))
#dot_W = 0
dot_L = theta*(lambda_*(A-x[:,1])-A)
#ax3.plot(t,num/denom,'k',alpha=0.5)
#ax3.plot(tau*t,xx[:,2], 'k-',label ='p')
dot_p = (x[:,2]* (1-x[:,3]) * dot_W / x[:,0] + eta*(w-x[:,0])*(E-A+x[:,1])+ w*dot_L ) / (1-x[:,3]*w-(1-x[:,3])*x[:,0])
dot_n = x[:,3]* (-(1-w)*dot_p/x[:,2] + 1/A*(eta*(E-A+x[:,1])+dot_L))

plt.figure()
T1,T2 = 3600,6332
T1,T2 = 3500,6332
plt.plot(t[T1:T2],L[T1:T2])
plt.plot(t[T1:T2],A[T1:T2]*(1-1/lambda_[T1:T2]))

plt.plot(t[T1:T2],p[T1:T2])
#num = (1-n)*rho*mu + eta*(w-W)*(E+L) - w *theta*lambda_*L
#dem =  (1-n)*rho + eta*n/w * (w-W) + theta*n*(lambda_-1)

num = (1-n)*rho*mu + eta*(w-W)*E
dem =  (1-n)*rho + eta*n/(w*lambda_) *(w-W) 
plt.plot(t[T1:T2],num[T1:T2]/dem[T1:T2])

#plt.plot(t[T1:T2],a0[T1:T2])
#plt.plot(t,dot_n / x[:,3])
#plt.plot(t[T1:T2],a1[T1:T2])
#plt.plot(t[T1:T2],theta*w*A[T1:T2]/ (p[T1:T2] * lambda_[T1:T2] * (1-n[T1:T2]*w-(1-n[T1:T2])*W[T1:T2])))

#plt.plot(t,theta*x[:,3]/lambda_ /(1-x[:,3]*w-(1-x[:,3])*x[:,0]))
plt.plot(t[T1:T2],lambda_[T1:T2]/10)



## Slow Manifold

##### - Implicit function countour method

In [410]:
W,L,p,n,s = np.transpose(x)
w=0.5

A = n*p/w
a1 = w/p * theta * (A - L) / (1-n*w-(1-n)*W)
a0 = (theta*n-(1-n)*rho*(mu/p-1)-eta/p*(w-W)*(E-A+L)) / (1-n*w-(1-n)*W)

tv,s_0,alpha =  0.1, 10**-6, 0.01
lambda_ = alpha * (s + s_0)**b

In [411]:
T1 = 1000
scale = 2
xmin, xmax, ymin, ymax, zmin, zmax = np.min([np.min(a0[T1:])/scale,np.min(a0[T1:])*scale]),np.max(a0[T1:])*scale, np.min([np.min(a1[T1:])/scale,np.min(a1[T1:])*scale]),np.max(a1[T1:])*scale,-1,np.max(lambda_[T1:])*1.1
#xmin, xmax, ymin, ymax = -0.1,0.1,-0.1,0.1

In [412]:
def plot_implicit(fn,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax ):
    ''' create a plot of an implicit function
    fn  ...implicit function (plot where fn==0)
    bbox ..the x,y,and z limits of plotted interval'''
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    A = np.linspace(xmin, xmax, 500) # resolution of the contour
    B = np.linspace(ymin, ymax, 100) # number of slices
    C = np.linspace(0, zmax, 100)
    A1,A2 = np.meshgrid(A,C) # grid on which the contour is plotted

    

    for y in B: # plot contours in the XZ plane
        X,Z = A1,A2
        Y = fn(X,y,Z,tv,s_0,alpha)
        cset = ax.contour(X, Y+y, Z, [y], zdir='y',alpha=0.2)


    # must set plot limits because the contour will likely extend
    # way beyond the displayed level.  Otherwise matplotlib extends the plot limits
    # to encompass all values in the contour.
    ax.set_zlim3d(zmin,zmax)
    ax.set_xlim3d(xmin,xmax)
    ax.set_ylim3d(ymin,ymax)

    plt.show()

In [413]:
def goursat_tangle(x,y,z,tv,s_0,alpha):
    #t,s_0,alpha = 1,10**-2,1
    return tv**2*(y*z-x)**2*z**2+s_0*z**2-alpha**2
    #return tv**2*(y*z-x)**2+s_0-alpha**2/z**2

#tv,s_0,alpha = 1,0.01,1
tv,s_0,alpha =  0.1, 10**-6, 0.01
#tv,s_0,alpha =  1, 0., 1

print('tv,s_0,alpha = ',tv,s_0,alpha)
plot_implicit(goursat_tangle,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax)

tv,s_0,alpha =  0.1 1e-06 0.01


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


In [414]:
T1,T2 = 15000,18000 #10

plt.plot(a0[T1:T2],a1[T1:T2],lambda_[T1:T2],'+--')
plt.plot(a0[0:1],a1[0:1],lambda_[0:1],'g+--')

plt.xlabel('a0')

Text(0.5, 0, 'a0')

In [90]:
#T1,T2 = 3610,4700 #10
#T1,T2 = 2280,4100 #100
#T1,T2 = 3600,5000 #1


T1,T2 = 3300,6000 #10

plt.plot(a0[T1:T2],a1[T1:T2],lambda_[T1:T2],'+--')

n_plus = (-b_+ d_ )/ (2*a_)
n_minus = (-b_ - d_ )/ (2*a_)

A_plus = A*n_plus/n
A_minus = A*n_minus/n

a0_plus =(theta*n_plus-(1-n_plus)*rho*(mu/p-1)-eta/p*(w-W)*(E-A_plus+L)) / (1-n_plus*w-(1-n_plus)*W)
a0_minus =(theta*n_minus-(1-n_minus)*rho*(mu/p-1)-eta/p*(w-W)*(E-A_minus+L)) / (1-n_minus*w-(1-n_minus)*W)

#a0_plus =(theta*n_plus)/ (1-n_plus*w-(1-n_plus)*W)
#a0_minus =(theta*n_minus) / (1-n*w-(1-n)*W)

#plt.plot(a0_plus[T1:T2],a1[T1:T2],lambda_[T1:T2],'.')
#plt.plot(a0_minus[T1:T2],a1[T1:T2],lambda_[T1:T2],'.')

#plt.plot(np.linspace(0,10,2700),np.linspace(0,10,2700)/lambda_[T1:T2],0*lambda_[T1:T2],'-')

T1,T2 = 4200,6500

plt.plot(a0[T1:T2],a0[T1:T2]/lambda_[T1:T2],lambda_[T1:T2],'-')

plt.xlabel('a0')

Text(0.5, 0, 'a0')

In [29]:
xmin, xmax, ymin, ymax, zmin, zmax = -1,10,-1,10,-1,10

# N  FASTTTT

In [25]:
scale = 1.5
T1= 1000
w=0.5
xmin, xmax, ymin, ymax, zmin, zmax = np.min(p[T1:])*(1/scale),np.max(p[T1:])*scale, np.min(L[T1:])*scale,np.max(L[T1:])*scale,-1,np.max(lambda_[T1:])*1.1


In [26]:
print(xmin, xmax, ymin, ymax, zmin, zmax)

11.053180509890227 46.012311060289306 -2.5830953785863175 24.463067590622856 -1 10.098448434227649


In [27]:
def plot_implicit(fn,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax ):
    ''' create a plot of an implicit function
    fn  ...implicit function (plot where fn==0)
    bbox ..the x,y,and z limits of plotted interval'''
    A = np.linspace(xmin, xmax, 200) # resolution of the contour
    B = np.linspace(ymin, ymax,200) # number of slices
    C = np.linspace(0, zmax, 200)
    A1,A2 = np.meshgrid(A,C) # grid on which the contour is plotted

    for z in B: # plot contours in the XY plane
        X,Y = A1,A2
        Z = fn(X,Y,z,tv,s_0,alpha)
        cset = ax.contour(X, Y, Z+z, [z], zdir='z',alpha=0.2)
        # [z] defines the only level to plot for this contour for this value of z


    for x in B: # plot contours in the YZ plane
        Y,Z = A1,A2
        X = fn(x,Y,Z,tv,s_0,alpha)
        cset = ax.contour(X+x, Y, Z, [x], zdir='x',alpha=0.2)

    for y in B: # plot contours in the XZ plane
        X,Z = A1,A2
        Y = fn(X,y,Z,tv,s_0,alpha)
        cset = ax.contour(X, Y+y, Z, [y], zdir='y',alpha=0.2)

 

    # must set plot limits because the contour will likely extend
    # way beyond the displayed level.  Otherwise matplotlib extends the plot limits
    # to encompass all values in the contour.
    ax.set_zlim3d(zmin,zmax)
    ax.set_xlim3d(xmin,xmax)
    ax.set_ylim3d(ymin,ymax)

    plt.show()

In [29]:
def goursat_tangle_plus(x,y,z,tv,s_0,alpha):
    #t,s_0,alpha = 1,10**-2,1*
    W=0.3
    a_ = eta*(w-W) - theta * (z-1)*(w-W)
    b_ = - eta*((E+y)/x *w*(w-W) + (w**2-2*w*W+W)/w) + theta * ((z-1)*(w-W) + w*y*z/ x *(w-W)) + (1-w)*rho*(mu/x-1)
    c_ = eta*(E+y)/x *(w**2-2*w*W+W) - theta * w * y *z/x * (w-W) - (1-w)*rho*(mu/x-1)
    DELTA = b_**2 - 4*a_*c_
    D_ = [i*(i>0) for i in DELTA]
    d_ = np.sqrt(D_)
    n = (-b_+ d_ )/ (2*a_)
    #n =  np.array([i *(i <3)*(i>-3) for i in n])
    carre = (1-n*w - (1-n)*W)
    return delta/2 * z * (1-z**2/alpha**2 * (s_0 + (tv*theta/carre * ((n- w*y/x)*z-n))**2))
    #return tv**2*(y*z-x)**2+s_0-alpha**2/z**2
    
def goursat_tangle_moins(x,y,z,tv,s_0,alpha):
    #t,s_0,alpha = 1,10**-2,1*
    W=0.3
    a_ = eta*(w-W) - theta * (z-1)*(w-W)
    b_ = - eta*((E+y)/x *w*(w-W) + (w**2-2*w*W+W)/w) + theta * ((z-1)*(w-W) + w*y*z/ x *(w-W)) + (1-w)*rho*(mu/x-1)
    c_ = eta*(E+y)/x *(w**2-2*w*W+W) - theta * w * y *z/x * (w-W) - (1-w)*rho*(mu/x-1)
    DELTA = b_**2 - 4*a_*c_
    D_ = [i*(i>0) for i in DELTA]
    d_ = np.sqrt(D_)
    n = (-b_- d_ )/ (2*a_)
    #n =  np.array([i *(i <3)*(i>-3) for i in n])
    carre = (1-n*w - (1-n)*W)
    return delta/2 * z * (1-z**2/alpha**2 * (s_0 + (tv*theta/carre * ((n- w*y/x)*z-n))**2))
    #return tv**2*(y*z-x)**2+s_0-alpha**2/z**2

#tv,s_0,alpha = 1,0.01,1
tv,s_0,alpha =  0.1, 10**-6, 0.01


print('tv,s_0,alpha = ',tv,s_0,alpha)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plot_implicit(goursat_tangle_plus,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax)#
#plot_implicit(goursat_tangle_moins,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax)

tv,s_0,alpha =  0.1 1e-06 0.01


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: No contour levels were found within the data range.
  del sys.path[0]
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: No contour levels were found within the data range.
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: No contour levels were found within the data range.


In [ ]:
#T1,T2 = 3610,4700 #10
#T1,T2 = 2280,4100 #100
#T1,T2 = 3600,5000 #1

#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
T1,T2 = 9500,15000 #10

plt.plot(p[T1:T2],L[T1:T2],lambda_[T1:T2],'+--')
plt.plot(p[0:1],L[0:1],lambda_[0:1],'g.')
plt.xlabel('p')

## 11/08 TRY (LAST TRY ?)

In [64]:
scale = 1.5
T1= 1000
w=0.5
xmin, xmax, ymin, ymax, zmin, zmax = np.min(p[T1:])*(1/scale),np.max(p[T1:])*scale, np.min(W[T1:])/scale,np.max(W[T1:])*scale,-1,np.max(lambda_[T1:])*1.1


In [65]:
print(xmin, xmax, ymin, ymax, zmin, zmax)

11.053180509890227 46.012311060289306 0.24194207380380325 0.6815938343000727 -1 10.098448434227649


In [12]:
def plot_implicit(fn,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax ):
    ''' create a plot of an implicit function
    fn  ...implicit function (plot where fn==0)
    bbox ..the x,y,and z limits of plotted interval'''
    A = np.linspace(xmin, xmax, 300) # resolution of the contour
    B = np.linspace(ymin, ymax,300) # number of slices
    C = np.linspace(zmin, zmax, 300)
    A1,A2 = np.meshgrid(A,C) # grid on which the contour is plotted

    

    for y in B: # plot contours in the XZ plane
        X,Z = A1,A2
        Y = fn(X,y,Z,tv,s_0,alpha)
        cset = ax.contour(X, Y+y, Z, [y], zdir='y',alpha=0.2)

 

    # must set plot limits because the contour will likely extend
    # way beyond the displayed level.  Otherwise matplotlib extends the plot limits
    # to encompass all values in the contour.
    ax.set_zlim3d(zmin,zmax)
    ax.set_xlim3d(xmin,xmax)
    ax.set_ylim3d(ymin,ymax)

    plt.show()

In [89]:
def goursat_tangle_plus(x,y,z,tv,s_0,alpha):
    #t,s_0,alpha = 1,10**-2,1*
    a_ = (1-w)*rho*(mu/x-1) + eta * (w-y)*(1-w)/ (z*w)
    b_ = -(1-w)*(rho*(mu/x-1)  + eta*(w-y)*E/x) - eta/z
    c_ = eta*E*w/x
    DELTA = b_**2 - 4*a_*c_
    D_ = [i*(i>0) for i in DELTA]
    d_ = np.sqrt(D_)
    n = (-b_+ d_ )/ (2*a_)
    A =x*n/w
    L = A*(1-1/z)

    a1 = w/x * theta * (A - L) / (1-n*w-(1-n)*y)
    a0 = (theta*n-(1-n)*rho*(mu/x-1)-eta/p*(w-y)*(E-A+L)) / (1-n*w-(1-n)*y)

    #n =  np.array([i *(i <3)*(i>-3) for i in n])
    return(z*(1-z**2/alpha**2 * (s_0 + tv**2*(a1*z-a0)**2)))
    #return(1-z**2/alpha**2 * (s_0 + (tv/(1-n*w - (1-n)*y) * ((1-n)*rho*(mu/x-1) + eta/x*(w-y)*(E-n*x/(w*z))))**2))
    #return tv**2*(y*z-x)**2+s_0-alpha**2/z**2
    
def goursat_tangle_moin(x,y,z,tv,s_0,alpha):
    #t,s_0,alpha = 1,10**-2,1*
    a_ = (1-w)*rho*(mu/x-1) + eta * (w-y)*(1-w)/ (z*w)
    b_ = -(1-w)*(rho*(mu/x-1)  + eta*(w-y)*E/x) - eta/z
    c_ = eta*E*w/x
    DELTA = b_**2 - 4*a_*c_
    D_ = [i*(i>0) for i in DELTA]
    d_ = np.sqrt(D_)
    n = (-b_- d_ )/ (2*a_)
    
    
    A =p*n/w
    L = A*(1-1/z)

    p = x
    W = y
    
    a1 = w/p * theta * (A - L) / (1-n*w-(1-n)*W)
    a0 = (theta*n-(1-n)*rho*(mu/p-1)-eta/p*(w-W)*(E-A+L)) / (1-n*w-(1-n)*W)
    #n =  np.array([i *(i <3)*(i>-3) for i in n])
    return(z*(1-z**2/alpha**2 * (s_0 + tv**2*(a1*z-a0)**2)))
    #return(1-z**2/alpha**2 * (s_0 + (tv/(1-n*w - (1-n)*y) * ((1-n)*rho*(mu/x-1) + eta/x*(w-y)*(E-n*x/(w*z))))**2))
    #return tv**2*(y*z-x)**2+s_0-alpha**2/z**2
    
    
#tv,s_0,alpha = 1,0.01,1
tv,s_0,alpha =  0.1, 10**-6, 0.01


print('tv,s_0,alpha = ',tv,s_0,alpha)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#plot_implicit(goursat_tangle_plus,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax)
plot_implicit(goursat_tangle_moin,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax)

tv,s_0,alpha =  0.1 1e-06 0.01


In [87]:
#T1,T2 = 3610,4700 #10
#T1,T2 = 2280,4100 #100
#T1,T2 = 3600,5000 #1

#fig = plt.figure()<


#ax = fig.add_subplot(111, projection='3d')
T1,T2 = 9500,15000 #10

plt.plot(p[T1:T2],W[T1:T2],lambda_[T1:T2],'+--')
plt.plot(p[0:1],W[0:1],lambda_[0:1],'g.')
plt.xlabel('p')

Text(0.05215269767243335, -0.060859025653183955, 'p')

In [9]:
## lambda and L

In [16]:
scale = 1.5
T1= 1000
w=0.5
xmin, xmax, ymin, ymax, zmin, zmax = np.min(p[T1:])*(1/scale),np.max(p[T1:])*scale, np.min(n[T1:])/scale,np.max(n[T1:])*scale,-1,np.max(lambda_[T1:])*1.1
ymin = -0.5

In [17]:
def goursat_tangle_moin(x,y,z,tv,s_0,alpha):
    #t,s_0,alpha = 1,10**-2,1*
    A =x*y/w
    L = A*(1-1/z)
    W = 0.4
    n = y
    p = x
    a1 = w/p * theta * (A - L) / (1-n*w-(1-n)*W)
    a0 = (theta*n-(1-n)*rho*(mu/p-1)-eta/p*(w-W)*(E-A+L)) / (1-n*w-(1-n)*W)
    #n =  np.array([i *(i <3)*(i>-3) for i in n])
    return(z*(1-z**2/alpha**2 * (s_0 + tv**2*(a1*z-a0)**2)))
    #return(1-z**2/alpha**2 * (s_0 + (tv/(1-n*w - (1-n)*y) * ((1-n)*rho*(mu/x-1) + eta/x*(w-y)*(E-n*x/(w*z))))**2))
    #return tv**2*(y*z-x)**2+s_0-alpha**2/z**2
    
    
#tv,s_0,alpha = 1,0.01,1
tv,s_0,alpha =  0.1, 10**-6, 0.01


print('tv,s_0,alpha = ',tv,s_0,alpha)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#plot_implicit(goursat_tangle_plus,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax)
plot_implicit(goursat_tangle_moin,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax)

tv,s_0,alpha =  0.1 1e-06 0.01


In [18]:
#T1,T2 = 3610,4700 #10
#T1,T2 = 2280,4100 #100
#T1,T2 = 3600,5000 #1

#fig = plt.figure()<


#ax = fig.add_subplot(111, projection='3d')
T1,T2 = 9500,15000 #10

plt.plot(p[T1:T2],n[T1:T2],lambda_[T1:T2],'+--')
plt.plot(p[0:1],n[0:1],lambda_[0:1],'g.')
plt.xlabel('p')

Text(-0.018105044898267187, -0.09266768761940895, 'p')

In [44]:

a_ = (1-w)*rho*(mu/p-1) + eta * (w-W)*(1-w)/ (lambda_*w)
b_ = -(1-w)*(rho*(mu/p-1)  + eta*(w-W)*E/p) - eta/lambda_
c_ = eta*E*w/p
DELTA = b_**2 - 4*a_*c_
D_ = [i*(i>0) for i in DELTA]
d_ = np.sqrt(D_)
nplus = (-b_+ d_ )/ (2*a_)
nmoins = (-b_- d_ )/ (2*a_)

In [53]:
plt.figure()

T1,T2 = 3600,17000
#plt.plot(t[T1:T2],nplus[T1:T2],'--')
plt.plot(t[T1:T2],nmoins[T1:T2],'--')
plt.plot(t[T1:T2],n[T1:T2])


In [ ]:
-b*delta*Y*(1-Y**2/alpha**2 * (s_0 + (tv/(1-nmoins*w - (1-nmoins)*W[(X/dt).astype(int)]) * ((1-nmoins)*rho*(mu/p[(X/dt).astype(int)]-1) + eta/p[(X/dt).astype(int)]*(w-W[(X/dt).astype(int)])*(E-nmoins*p[(X/dt).astype(int)]/(w*Y))))**2))


In [54]:
plt.figure()
d = 0.01
xrange = arange(t[T1], t[T2], dt)
yrange = arange(-1, 11, d)
X, Y = meshgrid(xrange,yrange)
# F is one side of the equation, G is the other
#F = delta*X**3 + X**2 * Y - delta/4 * Y**2  # Cusp with approximation

#F = -b*delta*Y*(1-Y**2/alpha**2 * (s_0 + tv**2*(a1[(X/dt).astype(int)]*Y-a0[(X/dt).astype(int)])**2))
A = n_
F = -b*delta*Y*(1-Y**2/alpha**2 * (s_0 + (tv/(1-nmoins[(X/dt).astype(int)]*w - (1-nmoins[(X/dt).astype(int)])*W[(X/dt).astype(int)]) * ((1-nmoins[(X/dt).astype(int)])*rho*(mu/p[(X/dt).astype(int)]-1) + eta/p[(X/dt).astype(int)]*(w-W[(X/dt).astype(int)])*(E-nmoins[(X/dt).astype(int)]*p[(X/dt).astype(int)]/(w*Y))))**2))


#plt.grid()
#ax5.contour(X, Y, F , [0])
plt.contour(X, Y, F , [0])
plt.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',alpha=0.5)

Text(0.5, 0, 'p')

In [240]:
x,y,z = p,L,lambda_
a_ = eta*(w-W) - theta * (z-1)*(w-W)
b_ = - eta*((E+y)/x *w*(w-W) + (w**2-2*w*W+W)/w) + theta * ((z-1)*(w-W) + w*y*z/ x *(w-W)) + (1-w)*rho*(mu/x-1)
c_ = eta*(E+y)/x *(w**2-2*w*W+W) - theta * w * y *z/x * (w-W) - (1-w)*rho*(mu/x-1)
DELTA = b_**2 - 4*a_*c_
D_ = [i*(i>0) for i in DELTA]
d_ = np.sqrt(D_)
n = (-b_+ d_ )/ (2*a_)
n = [i *(i <3)*(i>-3) for i in n]

In [241]:
plt.figure()
plt.plot(n)

Text(0.5, 0, 'p')

# L slow ma

In [25]:
W,L,p,n,s = np.transpose(x)
A = n*p/w

a1 = w/p * theta * (A - L) / (1-n*w-(1-n)*W)
a0 = (theta*n-(1-n)*rho*(mu/p-1)-eta/p*(w-W)*(E-A+L)) / (1-n*w-(1-n)*W)

tv,s_0,alpha =  0.1, 10**-6, 0.01
lambda_ = alpha * (s + s_0)**b

In [59]:
T1,T2 = 3000,30000
T1,T2 = 3610,5100
T1,T2 = 3010,5100

scale = 1.1
xmin, xmax, ymin, ymax, zmin, zmax = np.min(p[T1:])*(1/scale),np.max(p[T1:])*scale, np.min(n[T1:])*(1/scale),np.max(n[T1:])*scale,0.1,np.max(lambda_[T1:])*1.1
print(xmin, xmax, ymin, ymax, zmin, zmax)

15.047357672776052 33.751562588499716 0.007453993842569434 0.3309918229540748 0.1 10.091490754439295


In [60]:
np.mean(W)

0.42145849382072886

In [68]:
def plot_implicit(fn,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax ):
    ''' create a plot of an implicit function
    fn  ...implicit function (plot where fn==0)
    bbox ..the x,y,and z limits of plotted interval'''
    A = np.linspace(xmin, xmax, 100) # resolution of the contour
    B = np.linspace(ymin, ymax,100) # number of slices
    C = np.linspace(zmin, zmax, 100)
    A1,A2 = np.meshgrid(A,C) # grid on which the contour is plotted


    for y in B: # plot contours in the XZ plane
        X,Z = A1,A2
        Y = fn(X,y,Z,tv,s_0,alpha)
        cset = ax.contour(X, Y+y, Z, [y], zdir='y',alpha=0.2)

 

    # must set plot limits because the contour will likely extend
    # way beyond the displayed level.  Otherwise matplotlib extends the plot limits
    # to encompass all values in the contour.
    ax.set_zlim3d(zmin,zmax)
    ax.set_xlim3d(xmin,xmax)
    ax.set_ylim3d(ymin,ymax)

    plt.show()

In [75]:


def goursat_tangle(x,y,z,tv,s_0,alpha):
    #t,s_0,alpha = 1,10**-2,1*
    W=0.3
    
    #n =  np.array([i *(i <3)*(i>-3) for i in n])
    L = x*y /w * (1-z**-1)
    
    #L =  np.array([L *(L>-2) for i in L])
    return delta/2 * z * (1-z**2/alpha**2 * (s_0 + (tv*theta/(1-y*w - (1-y)*W) * ((y- w*L/x)*z-y))**2))
    #return tv**2*(y*z-x)**2+s_0-alpha**2/z**2

#tv,s_0,alpha = 1,0.01,1
tv,s_0,alpha =  0.1, 10**-6, 0.01


print('tv,s_0,alpha = ',tv,s_0,alpha)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#plot_implicit(goursat_tangle_plus,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax)
plot_implicit(goursat_tangle,tv,s_0,alpha,xmin, xmax, ymin, ymax, zmin, zmax)


tv,s_0,alpha =  0.1 1e-06 0.01


In [66]:


plt.plot(p[T1:T2],L[T1:T2],lambda_[T1:T2],'+--')
plt.plot(A[0:1],lambda_[0:1],L[0:1],'g.')
plt.xlabel('A')

Text(0.0005634247941578097, -0.08979996982597734, 'A')

other graph

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
T1,T2 = 3600,6332

plt.plot(a0[T1:T2],p[T1:T2],lambda_[T1:T2],'.',markersize = 0.4)
#ax.set_xlim3d(0,1)

In [176]:
plt.figure()

#T1,T2 = 3610,4100
T1,T2 = 3600,7500

plt.plot(a0[T1:T2],a1[T1:T2],'+--')
plt.plot(0*a0[T1:T2],a1[T1:T2],'+--')

In [145]:
np.argmin(a0[T1:T2])

71

In [283]:
T1,T2 = 2000,10000

plt.plot(a0[T1:T2],a1[T1:T2],lambda_[T1:T2],'.')

#plt.plot(a0[0:1],a1[0:1],lambda_[0:1],'g.')
plt.xlabel('a0')

Text(-0.03495305906203494, -0.09081071126310043, 'a0')

In [377]:

fig = plt.figure(figsize=(5,5))

ax1 = fig.add_subplot(3,1,1)
ax2 = fig.add_subplot(3,1,2)
ax3 = fig.add_subplot(3,1,3)

ax1.plot(t,lambda_)
ax2.plot(t,tv**2*(a1*lambda_ - a0)**2 + s_0 - (lambda_/alpha)**-2)
ax3.plot(t,a0)


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [165]:
plt.plot(theta*n / (1-n*w-(1-n)*W), theta*n/lambda_ /(1-n*w-(1-n)*W) ,lambda_)

In [166]:
plt.plot(theta*n / (1-n*w-(1-n)*W),w/p * theta*(A-L) /(1-n*w-(1-n)*W) ,lambda_)

In [81]:
plt.plot(theta*n / (1-n*w-(1-n)*W),a1,lambda_)

In [210]:
plt.figure()
dot_L = theta*(lambda_*(A-L)-A)
#plt.plot(t,dot_L)
plt.plot(t,a1)
plt.plot(t,w/p * theta*(E) /(1-n*w-(1-n)*W),alpha=0.5)

In [90]:
plt.figure()
plt.plot(t,(1-1/lambda_)*A,alpha=0.5)
plt.plot(t,L,alpha=0.5)

## $p$ slow or fast ?

In [62]:
#num = (1-n)*rho*mu+eta*(w-W)*(E+L)-w*theta*lambda_*L
#denom = (1-n)*rho + eta*(w-W)*n/w -theta*n*(lambda_-1)
num = (1-n)*rho*mu+eta*(w-W)*(E)
denom = (1-n)*rho - eta*(w-W)*n/ (w* lambda_)

A = x[:,2]*x[:,3]/w
mu = 25
#dot_W = x[0]*rho*((mu/x[2]-1)+float(np.random.normal(0,0.22,1)))
dot_W = x[:,0]*rho*((mu/x[:,2]-1))
#dot_W = 0
dot_L = theta*(lambda_*(A-x[:,1])-A)
#ax3.plot(t,num/denom,'k',alpha=0.5)
#ax3.plot(tau*t,xx[:,2], 'k-',label ='p')
dot_p = (x[:,2]* (1-x[:,3]) * dot_W / x[:,0] + eta*(w-x[:,0])*(E-A+x[:,1])+ w*dot_L ) / (1-x[:,3]*w-(1-x[:,3])*x[:,0])
dot_n = x[:,3]* (-(1-w)*dot_p/x[:,2] + 1/A*(eta*(E-A+x[:,1])+dot_L))

plt.figure()
T1,T2 = 3600,6332
T1,T2 = 3500,6332

plt.plot(t[T1:T2],p[T1:T2])

# L SLAVED
#num = (1-n)*rho*mu + eta*(w-W)*E
#dem =  (1-n)*rho + eta*n/(w*lambda_) *(w-W) 
#plt.plot(t[T1:T2],num[T1:T2]/dem[T1:T2],'--')

# L FREE
num = (1-n)*rho*mu + eta*(w-W)*(E+L)
dem =  (1-n)*rho + eta*n/w*(w-W) 

plt.plot(t[T1:T2],num[T1:T2]/dem[T1:T2],'--')

plt.plot(t[T1:T2],L[T1:T2])
plt.plot(t[T1:T2],(1-1/lambda_[T1:T2])*A[T1:T2],'--')


#plt.plot(t[T1:T2],(-a0[T1:T2]+a1[T1:T2]*lambda_[T1:T2])*p[T1:T2])
#plt.plot(t[T1:T2],dot_p[T1:T2])

#plt.plot(t[T1:T2],a0[T1:T2])
#plt.plot(t,dot_n / x[:,3])
#plt.plot(t[T1:T2],a1[T1:T2])
#plt.plot(t[T1:T2],theta*w*A[T1:T2]/ (p[T1:T2] * lambda_[T1:T2] * (1-n[T1:T2]*w-(1-n[T1:T2])*W[T1:T2])))

#plt.plot(t,theta*x[:,3]/lambda_ /(1-x[:,3]*w-(1-x[:,3])*x[:,0]))
#plt.plot(t[T1:T2],lambda_[T1:T2]/10)


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [206]:
# A- L and A /lambda


In [207]:
plt.figure()

plt.plot(t[T1:T2],A[T1:T2]-L[T1:T2])
plt.plot(t[T1:T2],A[T1:T2]/lambda_[T1:T2],'+--')

/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


# Compare $\dot{n}$ and $\dot{n}_{slow }$

In [367]:
a_ = (1-w)*rho*(mu/p-1)+ eta*(1-w)/(w*lambda_)
b_ = -((1-w)*(rho*(mu/p-1)+E*eta/lambda_) + eta/lambda_)
c_ = eta*w*E / p

In [368]:
DELTA = b_**2 - 4*a_*c_
D_ = [i*(i>0) for i in DELTA]
d_ = np.sqrt(D_)

In [369]:
dot_L = theta*(lambda_*(A-L)-A)
dot_p = ((1-n)*rho*(mu-p)+eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)
df3dn =  ( -rho*(mu-p)-(w-W)*eta*p/w + w*theta*(lambda_-1)*p/w ) / (1-w*n-(1-n)*W) - (-w+W)*((1-n)*rho*(mu-p)+eta*(E-A+L)+w*dot_L) / (1-w*n-(1-n)*W)**2

df4dn = w/p*(-eta*p/w+theta*(lambda_-1)*p/w)

dfndn = -(1-w)/p *(dot_p + n*df3dn) + df4dn

In [370]:
plt.figure()
T1,T2 = 3600,6332
T1,T2 = 3500,6632
size = 45
plt.plot(t[T1:T2],n[T1:T2]*size)
#plt.plot(t[T1:T2],n[T1:T2]-(-b_[T1:T2] - d_[T1:T2] )/ (2*a_[T1:T2]) )
#plt.plot(t[T1:T2],(-b_[T1:T2] + d_[T1:T2] )/ (2*a_[T1:T2])*size,'--')
plt.plot(t[T1:T2],(-b_[T1:T2] - d_[T1:T2] )/ (2*a_[T1:T2])*size,'--')
#plt.plot(t[T1:T2],(dfndn[T1:T2]>0))
#plt.plot(t[T1:T2],lambda_[T1:T2])


####  Compare $\dot{n}$ and $\dot{n}_{slow }$ (BIS)

In [371]:
a_ = eta*(w-W) - theta * (lambda_-1)*(w-W)
b_ = - eta*((E+L)/p *w*(w-W) + (w**2-2*w*W+W)/w) + theta * ((lambda_-1)*(w-W) + w*L*lambda_ / p *(w-W)) + (1-w)*rho*(mu/p-1)
c_ = eta*(E+L)/p *(w**2-2*w*W+W) - theta * w * L *lambda_ /p * (w-W) - (1-w)*rho*(mu/p-1)

In [372]:
DELTA = b_**2 - 4*a_*c_
D_ = [i*(i>0) for i in DELTA]
d_ = np.sqrt(D_)

In [373]:
plt.figure()

plt.plot(t[T1:T2],DELTA[T1:T2]>0)


In [374]:
plt.figure()
T1,T2 = 3600,6332
T1,T2 = 3500,6632


size = 45
plt.plot(t[T1:T2],n[T1:T2]*size)
#plt.plot(t[T1:T2],n[T1:T2]-(-b_[T1:T2] - d_[T1:T2] )/ (2*a_[T1:T2]) )
plt.plot(t[T1:T2],(-b_[T1:T2] + d_[T1:T2] )/ (2*a_[T1:T2])*size,'--')
plt.plot(t[T1:T2],(-b_[T1:T2] - d_[T1:T2] )/ (2*a_[T1:T2])*size,'--')

/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [260]:
# dot n slow manifold
plt.figure()
T1,T2 = 3000,6500

plt.plot(t[T1:T2],dot_n[T1:T2]/n[T1:T2])
dot_n_hat = w /p *eta *(E-A/lambda_)

num = (1-n)*rho*mu + eta*(w-W)*(E)
dem =  (1-n)*rho + eta*n*(w-W)/(w*lambda_)

dot_n_hat = dem*w / num *eta *E - eta*n/lambda_

plt.plot(t[T1:T2],dot_n_hat[T1:T2])
#plt.plot(t[T1:T2],dot_L[T1:T2])



/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


## P slow manifold

In [306]:
plt.figure()

plt.plot(t[T1:T2],p[T1:T2])


# L SLAVED
num = (1-n)*rho*mu + eta*(w-W)*(E)
dem =  (1-n)*rho + eta*n*(w-W)/(w*lambda_)

#plt.plot(t[T1:T2],num[T1:T2]/dem[T1:T2],'--')

# L FREE
num = (1-n)*rho*mu + eta*(w-W)*(E+L)
dem =  (1-n)*rho + eta*n*(w-W)/w

#plt.plot(t[T1:T2],num[T1:T2]/dem[T1:T2],'--')



# L FREE FREE
num = (1-n)*rho*mu + eta*(w-W)*(E+L) - w * theta*lambda_*L
dem =  (1-n)*rho + eta*n*(w-W)/w + theta*n*(1-lambda_)

plt.plot(t[T1:T2],num[T1:T2]/dem[T1:T2],'--')


#plt.plot(t[T1:T2],(dot_p[T1:T2])*20 ,'--')

plt.plot(t[T1:T2],L[T1:T2],'r')
plt.plot(t[T1:T2],(1-1/lambda_[T1:T2])*A[T1:T2],'r--')

/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [142]:
T1,T2 = 3400,6032

plt.figure()
#plt.plot(a0,lambda_)
plt.plot(a1[T1:T2],lambda_[T1:T2],'+--')
#plt.plot(a0[T1:T2],lambda_[T1:T2],'+--')

#plt.plot(a0,a0)

In [219]:
plt.figure()
#a1_hat = (w/p * theta * (A-L) )/(1-n*w-(1-n)*W)
a1_hat = (n*theta/lambda_)/(1-n*w-(1-n)*W)
plt.plot(t[T1:T2],a1_hat[T1:T2])
plt.plot(t[T1:T2],a1[T1:T2])

/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [ ]:
#######################################""

In [231]:
plt.figure()
T1,T2 = 3500,6632

#a1_hat = (w/p * theta * (A-L) )/(1-n*w-(1-n)*W)
L_hat = A*(1-1/lambda_)
plt.plot(t[T1:T2],L[T1:T2])
plt.plot(t[T1:T2],L_hat[T1:T2])


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [ ]:
################################################################""""""

In [186]:
plt.figure()
zz = L/p
plt.plot(t[T1:T2],zz[T1:T2])
plt.plot(t[T1:T2],lambda_[T1:T2])

/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [47]:
plt.figure()
T1,T2 = 12000, 22000
#a1_hat1 = (theta * n)/(1-n*w-(1-n)*W)
#a1_hat2 = (w/p * theta * (L) )/(1-n*w-(1-n)*W)

#a1_hat = (1-n*w-(1-n)*W)


#plt.plot(t[T1:T2],a1_hat[T1:T2])
#plt.plot(t[T1:T2],a1_hat2[T1:T2])
#plt.plot(t[T1:T2],a0[T1:T2],)
#
plt.plot(t[T1:T2],a1[T1:T2])
plt.plot(t[T1:T2],(5*1.8)*(A[T1:T2]-L[T1:T2])/p[T1:T2])

#plt.plot(t[T1:T2],a1_hat[T1:T2])



In [76]:
len(n[T1:T2])
T1 + np.argmin(n[T1:T2])


3677

1.0

In [77]:
lambda_ = alpha*(x[:,4]+s_0)**b


A = x[:,2]*x[:,3]/w
mu = 25

#dot_W = x[0]*rho*((mu/x[2]-1)+float(np.random.normal(0,0.22,1)))
dot_W = x[:,0]*rho*((mu/x[:,2]-1))
#dot_W = 0
dot_L = theta*(lambda_*(A-x[:,1])-A)
dot_p = (x[:,2]* (1-x[:,3]) * dot_W / x[:,0] + eta*(w-x[:,0])*(E-A+x[:,1])+ w*dot_L ) / (1-x[:,3]*w-(1-x[:,3])*x[:,0])
dot_n = x[:,3]* (-(1-w)*dot_p/x[:,2] + 1/A*(eta*(E-A+x[:,1])+dot_L))

ValueError: operands could not be broadcast together with shapes (30000,) (2,) 

In [124]:
plt.figure()
#plt.plot(a0,lambda_)
plt.plot(a1[T1:T2],lambda_[T1:T2],'+--')

#plt.plot(a0,a0)

In [20]:

plt.figure()



plt.plot(t,dot_n)
plt.plot(t,eta*(E*w/x[:,2] - x[:,3]/lambda_),alpha=0.3)
#plt.plot(t,w/x[:,2]*(eta*(E-A+x[:,1])),alpha=0.5)
#plt.plot(t,-x[:,3]*(1-w)*dot_p/x[:,2] + w/x[:,2]*(eta*(E-A+x[:,1])),alpha=0.2)

## $\dot{\lambda}$ and $ \dot{L}$

In [134]:
dot_L = theta*(lambda_*(A-L)-A)
dot_lambda = lambda_*delta*0.5*(1-(tv**2*(a1*lambda_-a0)**2 + s_0)*(alpha/lambda_)**-2)

In [158]:
fig = plt.figure()

T1,T2 = 3586,3700
#TT = 10
#T1,T2 = 3671-TT,3671+TT

plt.plot(t[T1:T2],dot_lambda[T1:T2])
plt.plot(t[T1:T2],dot_L[T1:T2])
plt.plot(t[T1:T2],lambda_[T1:T2])

#plt.plot(t[3671]+0*t[T1:T2],dot_L[T1:T2])


### Castastriphe in L ?


In [119]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
T1,T2 = 3600,6332

plt.plot(n[T1:T2],p[T1:T2],L[T1:T2],'+--',markersize = 0.4)
#ax.set_xlim3d(0,1)

TypeError: 'int' object is not subscriptable

In [ ]:
def goursat_tangle(x,y,z,tv,s_0,alpha):
    #t,s_0,alpha = 1,10**-2,1
    return tv**2*(y*z-x)**2*z**2+s_0*z**2-alpha**2
    #return tv**2*(y*z-x)**2+s_0-alpha**2/z**2

#tv,s_0,alpha = 1,0.01,1
tv,s_0,alpha =  0.1, 10**-6, 0.01

print('tv,s_0,alpha = ',tv,s_0,alpha)
plot_implicit(goursat_tangle,tv,s_0,alpha)

In [412]:
w = 0.5
W,L,p,n,s = np.transpose(x)
A = n*p/w
#a1 = w/p * theta * (A - L) / (1-n*w-(1-n)*W)
#a0 = (theta*n-(1-n)*rho*(mu/p-1)-eta/p*(w-W)*(E-A+L)) / (1-n*w-(1-n)*W)

tv,s_0,alpha =  0.1, 10**-6, 0.01
lambda_ = alpha * (s + s_0)**b

In [413]:
def min_max(l):
    return(np.min(l),np.max(l))

T1,T2 = 13200,20332
Wm,WM = min_max(W[T1:T2])

Lm,LM = min_max(L[T1:T2])
pm,pM = min_max(p[T1:T2])
nm,nM = min_max(n[T1:T2])
lambdam,lambdaM = min_max(lambda_[T1:T2])

Wampl = -Wm + WM
Lampl = -Lm + LM
pampl = -pm + pM
nampl = -nm + nM
lambdaampl  = -lambdam + lambdaM

In [414]:
ampl_v = [Wampl,Lampl ,pampl,nampl ,lambdaampl]
max_v = [WM,LM,pM,nM,lambdaM]

In [415]:
xampl = x/ampl_v
xmax = x/max_v

In [416]:
max_v

[4.201242474119821,
 -0.194763297045274,
 1.1734579852359721,
 -0.16053139529679458,
 0.10982162973895944]

In [423]:
scale_v

array([ 0.08367571, 18.02553989, 13.7827627 ,  0.29302766,  9.0576535 ])

# Eigenvalues

In [417]:
def eigenvalue(value,scale_v):

    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,10**-6,0.01,-0.5,25,0.1,0.5,0.1]
       #theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.01,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,lambda_ = value * scale_v
    
    Mat = np.zeros((5,5))
    A = n*p / w
    
    Mat[0,0] = rho*(mu/p-1)
    Mat[0,2] = -W*rho*mu/p**2

    Mat[1,1] = -theta*lambda_
    Mat[1,2] = theta*(lambda_-1)*n/w
    Mat[1,3] = theta*(lambda_-1)*p/w
    Mat[1,4] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)+eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    #df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) - (1-n)*dot_p / (1-n*w-(1-n)*W)**2
    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)**2 # change sign
    df3dl = ( (w-W)*eta + w*Mat[1,1] ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho)-(w-W)*eta*n/w + w*Mat[1,2] ) / (1-w*n-(1-n)*W)
    df3dn =  ( -rho*(mu-p)-(w-W)*eta*p/w + w*Mat[1,3] ) / (1-w*n-(1-n)*W) - (-w+W)*((1-n)*rho*(mu-p)+eta*(E-A+L)+w*dot_L) / (1-w*n-(1-n)*W)**2
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)

    Mat[2,0] = df3dW
    Mat[2,1] = df3dl
    Mat[2,2] = df3dp
    Mat[2,3] = df3dn
    Mat[2,4] = df3dlambda

    
    df4dl = w/p*(eta+Mat[1,1])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+Mat[1,2])
    df4dn = w/p*(-eta*p/w+Mat[1,3])
    df4dlambda = w/p*theta*(A-L)

    Mat[3,0] = -(1-w)*n/p * df3dW
    Mat[3,1] = -(1-w)*n/p * df3dl + df4dl
    Mat[3,2] = -(1-w)*n * (df3dp/p-dot_p/p**2) + df4dp
    Mat[3,3] = -(1-w)/p *(dot_p + n*df3dn) + df4dn
    Mat[3,4] = -(1-w)*n/p * df3dlambda + df4dlambda


    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))



    #Mat[4,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dW /p
    #Mat[4,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    #Mat[4,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    #Mat[4,3] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    #Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    Mat[4,0] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dW /p
    Mat[4,1] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dl /p
    Mat[4,2] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * (df3dp /p - dot_p/p**2)
    Mat[4,3] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * (df3dn /p)
    Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 - tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    P = np.zeros((5,5))
    for i in range(5):
        P[i,i] = 1/scale_v[i]
    
    Mat_2 = np.dot(np.dot(P,Mat),LA.inv(P))
    eigva, v = LA.eig(Mat_2)
    return(eigva,v)

In [418]:
start_time = time.time()
list_values = np.array([eigenvalue(value,scale_v)[0] for value in x])
print("--- %s seconds ---" % (time.time() - start_time))


--- 188.34072065353394 seconds ---


In [434]:
T1,T2 = 1600000,2000000
T1,T2 = 1000000,1900000
W,L,p,n,lambda_ = np.transpose(x)

In [435]:
xrange = arange(t[T1], t[T2], dt)

In [ ]:
fig = plt.figure(figsize=(8,8))


for i in range(5):
    print(i)
    fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
    ax1 = fig.add_subplot(3,1,1)
    ax2 = fig.add_subplot(3,1,2)
    ax3 = fig.add_subplot(3,1,3)
    
    #ax1.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',markersize=0.1)
        
    ax2.plot(t[T1:T2],list_values[:,i][T1:T2].real,'-',markersize=1,alpha=0.5)
    #ax2.plot(tt[T1:T2],0+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],-1+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],np.absolute(list_values[:,i][T1:T2]),'-',alpha=0.5)
    
    ax3.plot(t[T1:T2],list_values[:,i][T1:T2].imag,'.',markersize=0.5)

from numpy import arange
from numpy import meshgrid
d = 0.01
xrange = arange(t[T1], t[T2], dt)
yrange = arange(-1, 11, d)
X, Y = meshgrid(xrange,yrange)
# F is one side of the equation, G is the other
#F = delta*X**3 + X**2 * Y - delta/4 * Y**2  # Cusp with approximation

#F = -b*delta*Y*(1-Y**2/alpha**2 * (s_0 + tv**2*(a1[(X/dt).astype(int)]*Y-a0[(X/dt).astype(int)])**2))
#F = -b*delta*Y*(1-Y**2/alpha**2 * (s_0 + tv**2*(a1[(X/dt).astype(int)]*Y-a0[(X/dt).astype(int)])**2))

A = p*n/w

a1 = w/p * theta * (A - L) / (1-n*w-(1-n)*W)
a0 = (theta*n-(1-n)*rho*(mu/p-1)-eta/p*(w-W)*(E-A+L)) / (1-n*w-(1-n)*W)

G = -b*delta*Y*(1-Y**2/alpha**2 * (s_0 + tv**2*(a1[(X/dt).astype(int)]*Y-a0[(X/dt).astype(int)])**2))

ax1.contour(X, Y, G , [0])


ax1.set_ylabel('$\lambda$')        
ax2.set_ylabel('$Re(\lambda_i)$') 
#ax2.set_ylim([-20,5]) 

ax2.grid()
ax3.set_ylabel('$Imag(\lambda_i)$')         
ax3.set_xlabel('$t$')     

plt.show()

0
1
2


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  import sys
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reus

3
4


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  import sys
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reus

In [407]:
w,v = eigenvalue(x[int(1149980)],scale_v)
for i in range(len(w)):
    vect = v[:,i]
    vect = vect
    print(w[i],'\n',vect)

(-437.22842099596136+0j) 
 [-8.44964852e-06+0.j -2.19857338e-02+0.j -2.35070767e-02+0.j
 -1.70540530e-02+0.j  9.99336382e-01+0.j]
(0.22649031136926187+0j) 
 [-0.52496316+0.j -0.0775106 +0.j  0.81021031+0.j -0.24843378+0.j
  0.0156747 +0.j]
(-0.9075784070165275+0j) 
 [-0.16445424+0.j  0.10422218+0.j -0.93291714+0.j  0.30260106+0.j
 -0.01381151+0.j]
(-9.614153216388143+3.30053237139188j) 
 [-0.01023243-0.00351866j -0.57679791-0.12904546j -0.69882746+0.j
 -0.28345333+0.285528j    0.01448741-0.00962454j]
(-9.614153216388143-3.30053237139188j) 
 [-0.01023243+0.00351866j -0.57679791+0.12904546j -0.69882746-0.j
 -0.28345333-0.285528j    0.01448741+0.00962454j]


In [386]:
w,v = eigenvalue(x[int(1700500)],scale_v)
for i in range(len(w)):
    vect = v[:,i]
    vect = vect
    print(w[i],'\n',vect)

(-286.37291205239103+0j) 
 [-1.95793315e-05+0.j -3.28186657e-02+0.j -3.51127213e-02+0.j
 -2.56351574e-02+0.j  9.98515333e-01+0.j]
(0.23592591295598123+0j) 
 [-0.513275  +0.j -0.09847506+0.j  0.80784678+0.j -0.27216574+0.j
  0.01268183+0.j]
(-0.8626587216618975+0j) 
 [-0.17333651+0.j  0.12766894+0.j -0.91974833+0.j  0.32801426+0.j
 -0.01116876+0.j]
(-9.724924647004233+3.7014512549947214j) 
 [-0.01005964-0.00383492j -0.57751975-0.11044505j -0.70059746+0.j
 -0.30021972+0.27011122j  0.01216778-0.00895102j]
(-9.724924647004233-3.7014512549947214j) 
 [-0.01005964+0.00383492j -0.57751975+0.11044505j -0.70059746-0.j
 -0.30021972-0.27011122j  0.01216778+0.00895102j]


In [365]:
w,v = eigenvalue(x[int(1750000)],scale_v)
for i in range(len(w)):
    vect = v[:,i]
    vect = vect
    print(w[i],'\n',vect)

-0.1297541868239006 
 [ 0.92414587 -0.00097347  0.38165161 -0.01715379 -0.00111984]
-0.012495267691697023 
 [ 9.96512988e-01 -6.90736249e-05  8.33634541e-02 -3.51940888e-03
 -9.60863956e-05]
0.3150230050320131 
 [-0.35379889  0.44102247  0.29584035  0.62337982  0.45188673]
-11.151464432417944 
 [ 0.0161522   0.44635212  0.50665462  0.73742431 -0.00392046]
-9.076373395385286 
 [ 0.02225257  0.53716377  0.56832216  0.62285689 -0.00437322]


In [367]:
w,v = eigenvalue(x[int(1800000)],scale_v)
for i in range(len(w)):
    vect = v[:,i]
    vect = vect
    print(w[i],'\n',vect)

(0.40196909272048353+0j) 
 [-0.26687931+0.j  0.47721833+0.j  0.37731224+0.j  0.56147937+0.j
  0.49337054+0.j]
(-0.017714913022658187+0j) 
 [-9.97494868e-01+0.j  1.03617078e-03+0.j -7.02132313e-02+0.j
  8.48687403e-03+0.j  9.94621462e-04+0.j]
(-0.13056135138439498+0j) 
 [-0.90285852+0.j  0.01873908+0.j -0.42385876+0.j  0.06716212+0.j
  0.01812005+0.j]
(-10.866164631826559+0.8587849764271362j) 
 [-0.01374456+0.00156053j -0.43349831+0.10874937j -0.52353506+0.10172205j
 -0.71330572+0.j          0.08102082-0.01599006j]
(-10.866164631826559-0.8587849764271362j) 
 [-0.01374456-0.00156053j -0.43349831-0.10874937j -0.52353506-0.10172205j
 -0.71330572-0.j          0.08102082+0.01599006j]


In [374]:
w,v = eigenvalue(x[int(1850000)],scale_v)
for i in range(len(w)):
    vect = v[:,i]
    vect = vect
    print(w[i],'\n',vect)

-37.99238843198019 
 [ 0.00186795  0.27737143  0.31390276  0.29378745 -0.85919473]
-0.022197626350893944 
 [ 9.97918655e-01 -4.01329012e-05  6.31039411e-02 -1.32725007e-02
  2.98639289e-04]
-0.13241618456519946 
 [-0.87185322 -0.00506975 -0.4802617   0.09549617 -0.00868556]
-4.0466819442859245 
 [-0.03348501 -0.53436317 -0.59830634 -0.43630412 -0.40620561]
-9.383228983006036 
 [ 0.00658667  0.31902166  0.27319667 -0.13484206  0.89742019]


In [375]:
w,v = eigenvalue(x[int(1950000)],scale_v)
for i in range(len(w)):
    vect = v[:,i]
    vect = vect
    print(w[i],'\n',vect)

(-370.6495268255994+0j) 
 [-1.25357168e-05+0.j -2.66742057e-02+0.j -2.88105040e-02+0.j
 -2.13784363e-02+0.j  9.99000202e-01+0.j]
(-2.840050726199367+9.143008957822623j) 
 [-0.00321778-0.01042079j -0.6193867 +0.03280429j -0.64714247+0.j
 -0.42368574-0.06012575j -0.0308997 +0.11087001j]
(-2.840050726199367-9.143008957822623j) 
 [-0.00321778+0.01042079j -0.6193867 -0.03280429j -0.64714247-0.j
 -0.42368574+0.06012575j -0.0308997 -0.11087001j]
(-0.01980344121678737+0j) 
 [-9.99796612e-01+0.j  3.14100538e-04+0.j -1.84408065e-02+0.j
  8.15914064e-03+0.j -8.13881198e-06+0.j]
(-0.10836595547003745+0j) 
 [-0.84829913+0.j  0.01397265+0.j -0.4815081 +0.j  0.21986885+0.j
 -0.000998  +0.j]


In [373]:
v

array([[ 1.86795189e-03,  9.97918655e-01, -8.71853221e-01,
        -3.34850096e-02,  6.58666924e-03],
       [ 2.77371431e-01, -4.01329012e-05, -5.06975258e-03,
        -5.34363165e-01,  3.19021659e-01],
       [ 3.13902765e-01,  6.31039411e-02, -4.80261702e-01,
        -5.98306345e-01,  2.73196669e-01],
       [ 2.93787453e-01, -1.32725007e-02,  9.54961678e-02,
        -4.36304121e-01, -1.34842062e-01],
       [-8.59194732e-01,  2.98639289e-04, -8.68555612e-03,
        -4.06205605e-01,  8.97420189e-01]])

In [139]:
w,v = eigenvalue(x[int(3600)])
for i in range(len(w)):
    vect = v[:,i]/[0.26,Lampl,pampl,nampl,lambdaampl]
    #vect = v[:,i]
    vect = vect/np.linalg.norm(vect)
    print(w[i],'\n',vect)

(0.31690928509310523+0j) 
 [ 0.32787869+0.j  0.09468591+0.j -0.90783972+0.j  0.24356168+0.j
 -0.00590788+0.j]
(-0.7000921146858844+0j) 
 [ 0.14273985+0.j -0.11143162+0.j  0.94549351+0.j -0.27061473+0.j
  0.00424537+0.j]
(-9.905572541222917+3.4054118322130207j) 
 [-0.00584257-0.00148867j -0.51223891+0.j         -0.58225127+0.04733088j
 -0.56725538+0.27260407j -0.01478993-0.00279384j]
(-9.905572541222917-3.4054118322130207j) 
 [-0.00584257+0.00148867j -0.51223891+0.j         -0.58225127-0.04733088j
 -0.56725538-0.27260407j -0.01478993+0.00279384j]
(-5.718923342851809+0j) 
 [0.00858299+0.j 0.31206755+0.j 0.45469509+0.j 0.5888161 +0.j
 0.59083682+0.j]


In [133]:
w,v = eigenvalue(x[int(3640)])
for i in range(len(w)):
    vect = v[:,i]/[0.26,Lampl,pampl,nampl,lambdaampl]
    vect = vect/np.linalg.norm(vect)
    print(w[i],'\n',vect)

-10.633916407308297 
 [-7.40853340e-03 -4.12778802e-01 -4.49719132e-01 -7.92029971e-01
  1.83521371e-05]
-9.46492958692964 
 [-8.80924310e-03 -4.50727909e-01 -4.76226289e-01 -7.54967066e-01
  1.93795853e-05]
-0.1761494455347795 
 [ 6.17107408e-01 -4.12901148e-03  7.86437966e-01 -2.60139144e-02
 -1.38646916e-05]
0.05176607393417011 
 [-9.99783846e-01 -1.07482511e-04  2.07794295e-02 -6.81401208e-04
  2.06886908e-07]
0.2510826849601126 
 [-0.18543596  0.41450005  0.21388972  0.75067168  0.42958867]


In [109]:
T1,T2 = 3586,3593
plt.figure()

plt.plot(t[T1:T2],lambda_[T1:T2],'+--')

## fast eigenvalues

In [57]:
def fast_eigenvalue(value):

    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,10**-6,0.01,-0.5,25,0.1,0.5,0.1]
       #theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.01,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((2,2))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    

    Mat[0,0] = -theta*lambda_
    Mat[0,1] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)+eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    #df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) - (1-n)*dot_p / (1-n*w-(1-n)*W)**2
    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)**2 # change sign
    df3dl = ( (w-W)*eta + w*Mat[1,1] ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho)-(w-W)*eta*n/w + w*theta*(lambda_-1)*n/w ) / (1-w*n-(1-n)*W)
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)


    
    df4dl = w/p*(eta+Mat[0,0])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+theta*(lambda_-1)*n/w)
    df4dlambda = w/p*theta*(A-L)

    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))



    #Mat[4,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dW /p
    #Mat[4,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    #Mat[4,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    #Mat[4,3] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    #Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    Mat[1,0] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dl /p
    Mat[1,1] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 - tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    eigva, v = LA.eig(Mat)
    return(eigva,v)

In [58]:
list_values = np.array([fast_eigenvalue(value)[0] for value in x])

In [73]:
fig = plt.figure(figsize=(8,8))

T1,T2 = 3590,6232


for i in range(2):
    print(i)
    fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
    ax1 = fig.add_subplot(3,1,1)
    ax2 = fig.add_subplot(3,1,2)
    ax3 = fig.add_subplot(3,1,3)
    
    ax1.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',markersize=0.1)
        
    #ax2.plot(t[T1:T2],list_values[:,i][T1:T2].real,'+--',alpha=0.5)
    ax2.plot(t[T1:T2],list_values[:,i][T1:T2].real,'-',alpha=0.5)

    #ax2.plot(tt[T1:T2],0+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],-1+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],np.absolute(list_values[:,i][T1:T2]),'-',alpha=0.5)
    
    ax3.plot(t[T1:T2],list_values[:,i][T1:T2].imag,'.',markersize=0.5)

ax1.set_ylabel('$\lambda$')        
ax2.set_ylabel('$Re(\lambda_i)$') 
ax2.grid()
ax3.set_ylabel('$Imag(\lambda_i)$')         
ax3.set_xlabel('$t$')     

plt.show()

0
1


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if __name__ == '__main__':
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  # Remove the CWD from sys.path while we load stuff.
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: MatplotlibDeprecationWarning: Addi

In [60]:
w,v = fast_eigenvalue(x[int(3670)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

-1.384076733854978 
 [-9.99999778e-01 -6.66272588e-04]
0.2367471915016806 
 [-0.99667775 -0.0814461 ]


In [61]:
w,v = fast_eigenvalue(x[int(3677)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

-1.4079619125645009 
 [-9.99999758e-01 -6.95012703e-04]
0.23585406351220992 
 [-0.99658398 -0.08258558]


In [68]:
w,v = fast_eigenvalue(x[int(4000)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

-3.1323236672128782 
 [-0.99999924 -0.00123188]
0.21585882890536953 
 [-0.98609385 -0.16618939]


In [74]:
def fast_det(value):

    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,10**-6,0.01,-0.5,25,0.1,0.5,0.1]
       #theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.01,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((2,2))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    

    Mat[0,0] = -theta*lambda_
    Mat[0,1] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)+eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    #df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) - (1-n)*dot_p / (1-n*w-(1-n)*W)**2
    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)**2 # change sign
    df3dl = ( (w-W)*eta + w*Mat[1,1] ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho)-(w-W)*eta*n/w + w*theta*(lambda_-1)*n/w ) / (1-w*n-(1-n)*W)
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)


    
    df4dl = w/p*(eta+Mat[0,0])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+theta*(lambda_-1)*n/w)
    df4dlambda = w/p*theta*(A-L)

    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))



    #Mat[4,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dW /p
    #Mat[4,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    #Mat[4,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    #Mat[4,3] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    #Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    Mat[1,0] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dl /p
    Mat[1,1] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 - tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    return(np.linalg.det(Mat))

In [75]:
list_det = np.array([fast_det(value) for value in x])/10**7

In [76]:
fig = plt.figure(figsize=(8,8))

fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(3,1,1)
ax2 = fig.add_subplot(3,1,2)
ax3 = fig.add_subplot(3,1,3)

T1,T2 = 3540,4700

ax1.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',markersize=0.1)
#ax2.plot(t[T1:T2],0*list_det[T1:T2],'black',markersize=1,alpha=0.5)
ax2.plot(t[T1:T2],list_det[T1:T2],'+--',markersize=1,alpha=0.8)

ax3.plot(t[T1:T2],(list_det[T1:T2]>0),'+--',markersize=1,alpha=0.8)
